# Popular Data Science Questions
---
## Introduction
### What is this project about

In this scenario, we're working for a company that creates data science content, be it books, online articles, videos or interactive text-based platforms like [Dataquest](https://www.dataquest.io/). And as enthusiastic employees we've desided to o scour the internet in search for the answer to the question:
* **What is it that people want to learn about in data science?**

Thinking back to our experience when we first started learning programming, it occurs to us that if we wanted to figure out what programming content to write, we could consult [Stack Overflow](https://stackoverflow.com/) and see what kind of content is more popular. We've decided to investigate Stack Overflow a little more and found out that it is part of a question and answer website network called [Stack Exchange](https://stackexchange.com/).
![Image](https://dq-content.s3.amazonaws.com/469/se_sites.png)

---

Stack Exchange hosts a huge amount of sitess on a multitude of fields and subjects, including mathematics, physics, philosophy, and data science. Just what we are looking for!
Also we can include data engineering as our point of interest. So there are some websites relevant to our goal:
* [Data Science](https://datascience.stackexchange.com/)
* [Cross Validated](https://stats.stackexchange.com/) (a statistics site)
* [Artificial Intelligence](https://ai.stackexchange.com/)
* [Mathematics](https://math.stackexchange.com/)
* [Stack Overflow](https://stackoverflow.com/) itself
* [Database Administrators](https://dba.stackexchange.com/)
* [Unix & Linux](https://unix.stackexchange.com/)
* [Software Engineering](https://softwareengineering.stackexchange.com/)

### A closer look to the Stack Exchange websites
Let's take for example **Stack Overflow** and walk through it a bit just to get you familiar with it. Another sites are similar to Stack Overflow

Basically **it is Q&A website** where everyone welcomes to ask about a wide range of topics in computer programming. If someone runs into a problem, doesn't know exactly how a particular function works, or is struggling to get their code to work they come here.

Due to issues can be related to different programming languages, there is a **tagging system** that helps filter and navigate content. Also there is a **voting system** that allows rate best questions and answers so their authors can earn reputation points.

And finally there is a **job finding section** where you can try to get youself a new fancy job.

![Image](https://upload.wikimedia.org/wikipedia/commons/d/dd/Stack_Overflow_Home.png)

---
### Stack Exchange Database

Stack Exchange provides a public data base for each of its websites. Here's a link to query and explore [Data Science Stack Exchange's database](https://data.stackexchange.com/datascience/query/new). It uses a Transact-SQL to write queries.

There are a lot of tables. Here's the most interesting for our purpose:
* Posts
* Tags
* TagSynonymous
* Comments

## Probably start

In [1]:
import pandas as pd